# surfrad database interface

This database contains information about things that do not change all that frequently:

- instrument deployment
- instrument inventory

QC related databases, which change quit frequently,0 are disconnected to tis database as long as we are not having a central server that prevents simultanius writing.

In [90]:
import sqlite3

In [91]:
import surfradpy.database as srfdb

In [92]:
reload(srfdb)

<module 'surfradpy.database' from '/Users/htelg/prog/SURFRAD/surfradpy/database.py'>

In [94]:
p2db = 'surfrad_database.db'
db = srfdb.SurfradDatabase(p2db)

In [96]:
db.snapshot().keys()

dict_keys(['tables', 'schema', 'preview'])

In [97]:
db.snapshot()['tables']

['instrument_type',
 'channels_wl',
 'clearsky_fits',
 'sqlite_sequence',
 'sites',
 'instruments_mfrsr',
 'deployments']

In [99]:
db.dump_table('deployments')

,index,Date_start,Date_stop,Location,instrument_type_id,instrument_type,instrument_sn,facing
0,0,2014-06-23 00:00:00,2014-08-20 00:00:00,bao,1,mfrsr,453,unk
1,1,2014-09-15 00:00:00,NaN,ska,1,mfrsr,453,unk
2,2,2016-04-22 00:00:00,2018-10-15 00:00:00,mlo,1,mfrsr,453,unk
3,3,2018-10-16 00:00:00,NaN,cucf,1,mfrsr,453,unk
4,4,2020-03-12 00:00:00,2020-03-24 00:00:00,ska?,1,mfrsr,453,unk
...,...,...,...,...,...,...,...,...
273,4,2021-08-01 00:00:00,NaN,yes,1,mfrsr,688,unk
274,0,2020-12-15 00:00:00,NaN,2d401,1,mfrsr,689,unk
275,1,2021-03-01 00:00:00,2021-03-15 00:00:00,cucf,1,mfrsr,689,unk
276,3,2021-10-01 00:00:00,planned,kettle-ponds,1,mfrsr,689,unk


In [5]:
db.dump_table('sites')

,index,site_id,abb,name,elevation,latitude,longitude,network
0,0,1,tbl,Table Mountain,1689,40.12498,-105.23680,surfrad
1,1,2,bnd,Bondville,230,40.05192,-88.37309,surfrad
2,2,3,sxf,Sioux Falls,473,43.73403,-96.62328,surfrad
3,3,4,dra,Desert Rock,1007,36.62373,-116.01947,surfrad
4,4,5,fpk,Fort Peck,634,48.30783,-105.10170,surfrad
5,5,6,gwn,Goodwin Creek,98,34.25470,-89.87290,surfrad
6,6,7,psu,Penn. State Univ.,376,40.72012,-77.93085,surfrad


In [98]:
db.dump_table('instruments_mfrsr')

,index,Instrument,Place,Delivery,Logger,Logger_ID,Head_ID,Destination,Centroid_1625nm,Peak_1625nm,Filter_1625nm,Photodetector,Current_location,l_command,u_command,Location_1,OOB,Licor/Lamp,Field
0,1,632,mobile_1,2016-06-12 00:00:00,372.0,$B423,$5E86,mobile_1->GWN,1624.15,1623.597,1A,5476,GWN,L 15,U 10,"TCAP, RUT, WFIP2",6/28/2012; 10/15/2012,NaN,NaN
1,2,633,mobile_1,2016-06-12 00:00:00,373.0,$EBFB,$AD23,mobile_1->GWN,1623.14,1623.025,4A,5480,GWN,L -4,U -11,"TCAP, RUT, WFIP2",6/28/2012; 10/15/2012,NaN,NaN
2,3,634,ARM,2016-08-12 00:00:00,374.0,$C37E,$C287,ARM,NaN,NaN,NaN,NaN,SKA,L -2,U 5,NaN,2012-10-15 00:00:00,NaN,NaN
3,4,635,ARM,2016-08-12 00:00:00,375.0,$B351,$248C,ARM,NaN,NaN,NaN,NaN,SKA,L 7,U 48,NaN,2012-10-15 00:00:00,NaN,NaN
4,5,648,mobile_2,2016-01-14 00:00:00,390.0,$BF4F,$97A4,mobile_2,1623.58,1623.687,1B,6644,MLO,L 4,U -41,SLV,2016-09-26 00:00:00,NaN,NaN
5,6,649,mobile_2,2016-01-14 00:00:00,391.0,$C8C1,$19F2,mobile_2,1623.83,1623.889,5B,6645,SKA,L 4,U 32,SLV,2016-09-26 00:00:00,NaN,NaN
6,7,650,surfrad,2016-01-14 00:00:00,392.0,$EC88,$F3CE,FPK,1623.71,1623.766,7B,6646,FPK,L 4,U-21,PST,9/8/2015; 10/01/2016,2016-10-07 00:00:00,NaN
7,8,651,surfrad,2016-01-14 00:00:00,393.0,$F016,$ED2B,FPK,1623.97,1623.983,9B,6648,FPK,L 6,U-30,SXF,2016-10-01 00:00:00,2016-10-07 00:00:00,NaN
8,9,659,surfrad,2016-04-15 00:00:00,409.0,$D6EF,$3D6D,SXF,1623.68,1623.658,8B,NaN,SXF,L 5,U 31,NaN,6/6/2016,NaN,2016-06-06 00:00:00
9,10,660,surfrad,2016-04-15 00:00:00,410.0,$D824,$3126,TBL,1623.43,1623.506,10B,NaN,TBL,L 5,U ?,NaN,2015-06-19 00:00:00,NaN,NaN


## some useful commands

In [37]:
# delete table
db.execute_query('DROP TABLE IF EXISTS [what to delet];')

# update sites table

In [65]:
import atmPy.data_archives.NOAA_ESRL_GMD_GRAD.surfrad.surfrad as atmsrf

In [66]:
overwrite = True

In [77]:
df = db.dump_table('sites')
df = df.drop('index', axis=1)

In [78]:
# df = df.sort_values('site_id')

In [83]:
# create the table
####
for site in atmsrf._locations:
    abb = site['abbreviation'][0].lower()
    if (df.abb == abb).sum() > 0:
        if not overwrite:
            print(f'{abb} already in table ... skip')
            continue
        else:
            exists = df[df.abb == abb].iloc[0]
            site_id = exists.site_id
            index = exists.name
            df = df.drop(index)
    else:
        site_id  = df.site_id.max() + 1
        index = df.index.max() + 1
        
    
    row = dict(site_id = site_id,
               abb = abb,
               name = site['name'],
               network = 'surfrad',
               elevation = site['alt'],
               latitude = site['lat'],
               longitude = site['lon'],
              )
    row
         
    
    df = pd.concat([df,pd.DataFrame(row, index=[index])])

    ######
    # below are the cases where there are alternative names
    if row['abb'] == 'bnd':
        row['abb'] = 'bon'
        df = pd.concat([df,pd.DataFrame(row, index=[index])])
    if row['abb'] == 'fpk':
        row['abb'] = 'fpe'
        df = pd.concat([df,pd.DataFrame(row, index=[index])])

df = df.sort_values('site_id')

In [84]:
df

,site_id,abb,name,elevation,latitude,longitude,network
0,1,tbl,Table Mountain,1689,40.12498,-105.23680,surfrad
1,2,bnd,Bondville,230,40.05192,-88.37309,surfrad
1,2,bon,Bondville,230,40.05192,-88.37309,surfrad
2,3,sxf,Sioux Falls,473,43.73403,-96.62328,surfrad
3,4,dra,Desert Rock,1007,36.62373,-116.01947,surfrad
4,5,fpk,Fort Peck,634,48.30783,-105.10170,surfrad
4,5,fpe,Fort Peck,634,48.30783,-105.10170,surfrad
5,6,gwn,Goodwin Creek,98,34.25470,-89.87290,surfrad
6,7,psu,Penn. State Univ.,376,40.72012,-77.93085,surfrad


In [87]:
# save into database
#####
if 0:
    with sqlite3.connect(db.path2db) as dbt:
        df.to_sql('sites', dbt, if_exists='replace')

# update MFRSR_History

In [3]:
import surfradpy.database as srfdb

In [46]:
reload(srfdb)

<module 'surfradpy.database' from '/Users/htelg/prog/SURFRAD/surfradpy/database.py'>

In [88]:
p2db = 'surfrad_database.db'
db = srfdb.SurfradDatabase(p2db)

In [89]:
out = db.update_mfrsr_history('MFRSR_History.xlsx', 
                              replace=True,
                              # dryrun=True
                             )

Instrument 451 has fist column with the name Purchased SN 451 w/stand from eBay July 2024 not "Date_start" as expected. This indicates that the instrument was never put into action ... bought for spare parts?
Instrument 607 has fist column with the name Brookhaven MFRSR not "Date_start" as expected. This indicates that the instrument was never put into action ... bought for spare parts?
Instrument 559 has fist column with the name Thermopile MFRSR (Standard filters) not "Date_start" as expected. This indicates that the instrument was never put into action ... bought for spare parts?
Instrument 584 has fist column with the name MFRSR purchased around 2013 not "Date_start" as expected. This indicates that the instrument was never put into action ... bought for spare parts?


In [19]:
dft = db.tp_df.iloc[[0]]

In [21]:
dft.Date_start

0   2014-06-23
Name: Date_start, dtype: datetime64[us]

In [23]:
dft.Date_start.dtypes.name == 'datetime64[us]'

'datetime64[us]'